[Open in Colab](https://colab.research.google.com/drive/1B9MrKiU9UvNhyOt0yZKVRloGEX0eak2Z)

In [0]:
# !pip freeze # requirements.txt or pipfile

In [0]:
# requirements for notebook
!pip install pandas gspread gspread-dataframe

In [0]:
import os
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.client import GoogleCredentials

In [0]:
BASE_DIR = "/content/drive"
DRIVE_DIR = os.path.join(BASE_DIR, 'My Drive')
BACKUP_DIR = os.path.join(DRIVE_DIR, 'backup')
BEFORE_CHANGES_CSV = os.path.join(BACKUP_DIR, 'before.csv')
AFTER_CHANGES_CSV = os.path.join(BACKUP_DIR, 'after.csv')
os.makedirs(BACKUP_DIR, exist_ok=True)

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from google.colab import drive
drive.mount(BASE_DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
default_app = GoogleCredentials.get_application_default()

In [0]:
gc = gspread.authorize(default_app)

In [0]:
SPREADSHEET_NAME = 'BASE CFE'
PRIMARY_WORKSHEET = "Primary"
INPUT_WORKSHEET = "Input"
OUTPUT_WORKSHEET = 'Output'
SETUP_SPREADSHEET = False

In [0]:
if SETUP_SPREADSHEET:
  spreadsheet = gc.create(SPREADSHEET_NAME)
  spreadsheet.add_worksheet(INPUT_WORKSHEET, 1, 1)
  spreadsheet.add_worksheet(OUTPUT_WORKSHEET, 1, 1)

In [0]:
spreadsheet = gc.open(SPREADSHEET_NAME)

In [0]:
worksheet = spreadsheet.worksheet(title=PRIMARY_WORKSHEET)
worksheet_in = spreadsheet.worksheet(title=INPUT_WORKSHEET)
worksheet_out = spreadsheet.worksheet(title=OUTPUT_WORKSHEET)

In [0]:
rows = worksheet.get_all_values()
print(rows[:5])

[['Title', 'Rank', 'Views'], ['John Wick', 'A', '100000'], ['John Wick 2', 'A', '5000'], ['John Wick 3', 'A+', '124201']]


In [0]:
df = pd.DataFrame.from_records(rows)
# df.rename(columns=df.iloc[0], inplace=True)
# df.drop(df.index[0], inplace=True)
df.head()

,0,1,2
0,Title,Rank,Views
1,John Wick,A,100000
2,John Wick 2,A,5000
3,John Wick 3,A+,124201


In [0]:
# set the dataframe column names based on the first row, and dropping that first row
df = df.rename(columns=df.iloc[0]).drop(df.index[0])
df.head()

,Title,Rank,Views
1,John Wick,A,100000
2,John Wick 2,A,5000
3,John Wick 3,A+,124201


In [0]:
df.to_csv(BEFORE_CHANGES_CSV, index=False)

In [0]:
df['By_Thousand'] = df['Views']
df.head()

,Title,Rank,Views,By_Thousand
1,John Wick,A,100000,100000
2,John Wick 2,A,5000,5000
3,John Wick 3,A+,124201,124201


In [0]:
df['Views'] = df['Views'].astype(int)
df['By_Thousand'] = df['Views'] * 0.001
df.head()

,Title,Rank,Views,By_Thousand
1,John Wick,A,100000,100.000
2,John Wick 2,A,5000,5.000
3,John Wick 3,A+,124201,124.201


In [0]:
df.to_csv(AFTER_CHANGES_CSV, index=False)

In [0]:
set_with_dataframe(worksheet_out, df)

In [0]:
from_csv_df = pd.read_csv(AFTER_CHANGES_CSV)
from_csv_df.head()

,Title,Rank,Views,By_Thousand
0,John Wick,A,100000,100.000
1,John Wick 2,A,5000,5.000
2,John Wick 3,A+,124201,124.201


In [0]:
from_csv_df.dtypes

Title           object
Rank            object
Views            int64
By_Thousand    float64
dtype: object